# Training Notebook

###  NOTE: Please run on Python 2.7!!

Pull labeled sensor data from the server, train a classification model on it, and export to Core ML.

In [1]:
import numpy as np
import pandas as pd
import turicreate as tc

/anaconda3/envs/py27/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Constants

In [21]:
FEATURES = ['AccelerationX', 'AccelerationY', 'AccelerationZ']
ACTIVITIES = ['activity1', 'activity2']
WINDOW_SIZE = 100
N = 10000
SESSION_SIZE = 1000
LABEL = 'Label'
SESSION = 'Session'

MU = 0
SIGMA = 5
np.random.seed(0)

In [22]:
# Method for generating ground truth prediction, given features.
def get_prediction(df):
    avg = np.mean(np.array(df, dtype = float))
    if avg > 0:
        return ACTIVITIES[0]
    else:
        return ACTIVITIES[1]

### Step 1: Generate synthetic sensor data
Labels are stored as strings (such as 'activity1', 'activity2', etc.)

In [23]:
# Generate random feature data
df = pd.DataFrame()
for feature in FEATURES:
    df[feature] = np.random.normal(loc=MU,scale=SIGMA,size=N)

# Set blank session and labels.
df[LABEL] = [ACTIVITIES[0]]*N
df[SESSION] = [0]*N

In [30]:
# Set the actual session values.
for s in range(N/SESSION_SIZE):
    s_start = s*SESSION_SIZE
    s_end = (s+1)*SESSION_SIZE
    df.loc[s_start:s_end, SESSION] = s

# Set the actual activity class labels
for w in range(N/WINDOW_SIZE):
    w_start = w*WINDOW_SIZE
    w_end = (w+1)*WINDOW_SIZE
    
    indicator = np.random.uniform(size=1)[0] > 0.5
    if indicator:
        df.loc[w_start:w_end, FEATURES] += 1.5
    else:
        df.loc[w_start:w_end, FEATURES] -= 1.5

    df.loc[w_start:w_end, LABEL] = get_prediction(df.loc[w_start:w_end, FEATURES])

### Step 2: Export sample data for in-app testing

### Note: You will have to drag this csv into the Xcode Project to update the sample data

In [31]:
df.loc[:999,:].to_csv("sample_data.csv")

## Run the model

### Step 1: Split data into train and test sets

In [32]:
data_tc = tc.SFrame(df)

In [33]:
seed = 120
train, valid = tc.activity_classifier.util.random_split_by_session(data_tc, session_id = SESSION, 
                                                                  fraction = 0.8, seed = seed)

### Step 2: Build the model using turicreate

In [34]:
model_100 = tc.activity_classifier.create(train, session_id = SESSION, target = LABEL, 
                                          features = FEATURES,  prediction_window = WINDOW_SIZE, 
                                          validation_set = valid)

Pre-processing 8000 samples...

Using sequences of size 2000 for model creation.

Processed a total of 8 sessions.

Using GPU to create model (AMD Radeon Pro 460)
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| Iteration           | Train Accuracy      | Train Loss          | Validation Accuracy | Validation Loss     | Elapsed Time        |
+---------------------+---------------------+---------------------+---------------------+---------------------+---------------------+
| 1                   | 0.587               | 1.026               | 0.850               | 0.298               | 0.1                 | 
| 2                   | 0.775               | 0.410               | 0.900               | 0.254               | 0.2                 | 
| 3                   | 0.762               | 0.387               | 0.900               | 0.233               | 0.3                 | 
| 4                   | 0.850               | 0.170               | 0.900               | 0.220               | 0.4                 | 
| 5        

### Step 3: Export in Core ML format

### Note: You will have to drag this mlmodel into the Xcode Project to update the model

In [35]:
filename = 'ActivityClassifier'
model_100.name = 'Activity Classifier'
model_100.export_coreml(filename + '.mlmodel')
model_100.save(filename)